## In this notebook we will perform inference wheater embeddings vector represents subgraph of normal or  anomalous transaction. using our anomaly detection model     
---
**NOTE**: 

In real life scenarios financial transaction are dynamically evolving graphs. Performing anomaly detection inference on graph embeddings in live Transaction Monitoring Systems will require 1st to update the graph and node representations after new transactions arrive. Recomputing entire graph for every newly arrived transaction will lead to unaxeptable delayes and even monitoring system failure. This problem  will be more sever if large amount of updates happen in a short time window.

Contact us at Logical Clocks and we will help you to setup end to end graph based deep anomaly detection live Transaction Monitoring Systems. 

---

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
34,application_1612198280619_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import json
import numpy as np
from hops import model
from hops.model import Metric

# Query Model Repository for best anomaly detection model

In [9]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [10]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [11]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ganAml
Model version: 1
{'loss': '0.00033020367845892906'}

# Create Model Serving of Exported Model

In [12]:
from hops import serving

In [13]:
MODEL_NAME

'ganAml'

In [17]:
best_model['version']

1

In [19]:
model_path="/Models/" + best_model['name']
model_path

'/Models/ganAml'

In [14]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

An error was encountered:
Could not create or update serving (url: /hopsworks-api/api/project/119/serving/), server response: 
 HTTP code: 422, HTTP reason: UNPROCESSABLE_ENTITY, error code: 120001, error msg: An argument was not provided or it was malformed., user msg: The model path does not respect the TensorFlow standard
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 193, in create_or_update
    topic_name, num_partitions, num_replicas, serving_id, instances)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 304, in _create_or_update_serving_rest
    error_code, error_msg, user_msg))
hops.exceptions.RestAPIError: Could not create or update serving (url: /hopsworks-api/api/project/119/serving/), server response: 
 HTTP code: 422, HTTP reason: UNPROCESSABLE_ENTITY, error code: 120001, error msg: An argument was not provided or it was malformed., user msg: The model pa

In [15]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

In [16]:
# Get serving status
serving.get_status(MODEL_NAME)

An error was encountered:
No serving with name: ganAml could be found among the list of available servings: 
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 425, in get_status
    serving = _find_serving_with_name(serving_name, servings)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 466, in _find_serving_with_name
    "available servings: {}".format(serving_name, serving_names_str))
hops.serving.ServingNotFound: No serving with name: ganAml could be found among the list of available servings: 



# Start Model Serving Server

In [11]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

Starting serving with name: ganAml...
Serving with name: ganAml successfully started

In [12]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [13]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [14]:
eval_td = fs.get_training_dataset("gan_eval_df", 1)

In [15]:
eval_td.show(5)

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     1|[-0.8758349, 1.15...|
|     1|[0.30752838, -0.1...|
|     1|[-0.49007943, -0....|
|     1|[-0.46613508, -0....|
|     1|[-0.9974007, 0.32...|
+------+--------------------+
only showing top 5 rows

In [25]:
def model_server(model_name, input):
    data = {"signature_name": "serving_default", "inputs": [input]}
    return serving.make_inference_request(model_name, data)['outputs']

In [39]:
scored_df = eval_td.read()\
                   .rdd.map(lambda x: (x.target,model_server(MODEL_NAME, np.array(x.embedding).tolist()))).map(lambda f: (f[0],f[1][0]))\
                   .toDF().toDF("target","score")

In [40]:
scored_df.show()

+------+----------+
|target|     score|
+------+----------+
|     1|8.52341175|
|     1|4.68761492|
|     1|5.79249573|
|     1|5.91423655|
|     1|7.08572435|
|     1|2.45506454|
|     1|3.70257711|
|     1|4.33131218|
|     1|8.08009243|
|     1|7.95012951|
|     1|6.53424883|
|     1|8.20870209|
|     1|6.20180655|
|     1|7.03330612|
|     1|6.28016472|
|     1|   4.55949|
|     1|5.02041245|
|     1|5.51366901|
|     1| 4.8042531|
|     1|4.64139605|
+------+----------+
only showing top 20 rows